In [ ]:
# Fresh reload - restart kernel then run this first
import importlib
import sys

# Clear any cached modules
modules_to_clear = [k for k in sys.modules.keys() if k.startswith('nirs4all')]
for module in modules_to_clear:
    if module in sys.modules:
        del sys.modules[module]

# Now reimport everything
from nirs4all.dataset.predictions import Predictions
from nirs4all.pipeline.runner import PipelineRunner
from nirs4all.pipeline.config import PipelineConfig
from nirs4all.dataset import dataset
from sample import dataset_config, pipeline_config
from nirs4all.dataset.loader import get_dataset
from nirs4all.controllers.registry import reset_registry
from nirs4all.controllers import *

# Test the new predictions class first
test_predictions = Predictions()
print(f"Empty predictions: {test_predictions}")
print(f"Length: {len(test_predictions)}")

# Test adding a prediction
import numpy as np
test_predictions.add_prediction(
    dataset="test",
    pipeline="test_pipe",
    model="TestModel",
    partition="test",
    y_true=np.array([1.0, 2.0]),
    y_pred=np.array([1.1, 2.1])
)

print(f"After adding: {test_predictions}")
print("✅ Predictions class is working correctly!")

print("\n" + "="*50)
print("Now running the actual pipeline...")

# Run the pipeline
data = get_dataset(dataset_config)
config = PipelineConfig(pipeline_config, "demo_pipeline")

runner = PipelineRunner()
try:
    res_dataset, history, pipeline = runner.run(config, data)
    print(f"\nPipeline completed successfully!")
    print(f"Final dataset predictions: {res_dataset._predictions}")
except Exception as e:
    print(f"Pipeline failed: {e}")
    import traceback
    traceback.print_exc()

Registering controller: DummyController
Registering controller: SklearnModelController
Registering controller: TensorFlowModelController
Registering controller: PyTorchModelController
Registering controller: TransformerMixinController
Registering controller: YTransformerMixinController
Registering controller: FeatureAugmentationController
Registering controller: SampleAugmentationController
Registering controller: CrossValidatorController
Registering controller: SpectraChartController
Registering controller: FoldChartController
Registering controller: YChartController
Empty predictions: 📈 Predictions: No predictions stored
Length: 0
After adding: 📈 Predictions: 1 entries
   Datasets: ['test']
   Pipelines: ['test_pipe']
   Models: ['TestModel']
✅ Predictions class is working correctly!

Now running the actual pipeline...
Loading dataset:
------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Check the pipeline results and predictions
print("Pipeline Results Analysis:")
print("="*50)

print(f"Final dataset: {res_dataset}")
print(f"\nPredictions stored: {res_dataset._predictions}")
print(f"Number of predictions: {len(res_dataset._predictions)}")

if len(res_dataset._predictions) > 0:
    print(f"Prediction keys: {res_dataset._predictions.list_keys()}")
    print(f"Datasets: {res_dataset._predictions.list_datasets()}")
    print(f"Pipelines: {res_dataset._predictions.list_pipelines()}")
    print(f"Models: {res_dataset._predictions.list_models()}")
    print(f"Partitions: {res_dataset._predictions.list_partitions()}")

    # Get details of first prediction
    keys = res_dataset._predictions.list_keys()
    if keys:
        first_key = keys[0]
        first_pred_parts = first_key.split('_', 3)  # Split into 4 parts max
        if len(first_pred_parts) >= 4:
            dataset_name, pipeline_name, model_name, partition_name = first_pred_parts
            first_pred = res_dataset._predictions.get_prediction_data(
                dataset_name, pipeline_name, model_name, partition_name
            )
            if first_pred:
                print(f"\nFirst prediction details:")
                print(f"  Dataset: {first_pred['dataset']}")
                print(f"  Pipeline: {first_pred['pipeline']}")
                print(f"  Model: {first_pred['model']}")
                print(f"  Partition: {first_pred['partition']}")
                print(f"  Y_true shape: {first_pred['y_true'].shape}")
                print(f"  Y_pred shape: {first_pred['y_pred'].shape}")
                print(f"  Sample indices: {len(first_pred['sample_indices'])} samples")
                print(f"  Fold index: {first_pred['fold_idx']}")
                print(f"  Metadata: {first_pred['metadata']}")
else:
    print("⚠️ No predictions were stored during pipeline execution")
    print("This might be because:")
    print("1. The prediction storage integration is not working correctly")
    print("2. The dataset parameter is not being passed properly")
    print("3. There were errors in the model training that prevented prediction storage")

Pipeline Results Analysis:
Final dataset: 📊 Dataset: sample_data
Features (samples=189, sources=1):
- Source 0: (189, 3, 2151), processings=['raw_MinMaxScaler_1', 'raw_MinMaxScaler_1_Gaussian_2', 'raw_MinMaxScaler_1_StandardScaler_3_Haar_4'], min=-0.215, max=0.89, mean=0.22, var=0.102)
Targets: (samples=189, targets=1, processings=['numeric', 'numeric_StandardScaler5'])
- numeric: min=1.33, max=128.31, mean=30.779
- numeric_StandardScaler5: min=-1.23, max=4.156, mean=0.019
Indexes:
- "train", ['raw_MinMaxScaler_1', 'raw_MinMaxScaler_1_Gaussian_2', 'raw_MinMaxScaler_1_StandardScaler_3_Haar_4']: 130 samples
- "test", ['raw_MinMaxScaler_1', 'raw_MinMaxScaler_1_Gaussian_2', 'raw_MinMaxScaler_1_StandardScaler_3_Haar_4']: 59 samples
Folds: [(97, 33), (97, 33), (97, 33)]

Predictions stored: 📈 Predictions: 8 entries
   Datasets: ['sample_data']
   Pipelines: ['config_demo_pipeline_00090c']
   Models: ['PLSRegression', 'function', 'Sequential']
Number of predictions: 8
Prediction keys: ['sampl

In [ ]:
# Demonstrate prediction analysis capabilities
print("Prediction Analysis Example:")
print("="*50)

# Get all predictions from PLSRegression model
pls_predictions = res_dataset._predictions.get_predictions(model="PLSRegression")
print(f"PLSRegression predictions: {list(pls_predictions.keys())}")

# Get specific test set prediction for PLS
pls_test = res_dataset._predictions.get_prediction_data(
    "sample_data", "config_demo_pipeline_00090c", "PLSRegression", "test"
)

if pls_test:
    import numpy as np
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

    y_true = pls_test['y_true'].flatten()
    y_pred = pls_test['y_pred'].flatten()

    print(f"\nPLS Test Set Performance:")
    print(f"  MSE: {mean_squared_error(y_true, y_pred):.4f}")
    print(f"  MAE: {mean_absolute_error(y_true, y_pred):.4f}")
    print(f"  R²:  {r2_score(y_true, y_pred):.4f}")
    print(f"  Y processing: {pls_test['metadata']['y_processing']}")

# Combine cross-validation folds for PLS
pls_cv_combined = res_dataset._predictions.combine_folds(
    "sample_data", "config_demo_pipeline_00090c", "PLSRegression", "test_fold"
)

if pls_cv_combined:
    y_true_cv = pls_cv_combined['y_true'].flatten()
    y_pred_cv = pls_cv_combined['y_pred'].flatten()

    print(f"\nPLS Cross-Validation Combined Performance:")
    print(f"  Samples: {len(y_true_cv)}")
    print(f"  MSE: {mean_squared_error(y_true_cv, y_pred_cv):.4f}")
    print(f"  MAE: {mean_absolute_error(y_true_cv, y_pred_cv):.4f}")
    print(f"  R²:  {r2_score(y_true_cv, y_pred_cv):.4f}")
    print(f"  Folds: {pls_cv_combined['metadata']['num_folds']}")

# Compare all models on test set
print(f"\nModel Comparison (Test Set):")
models = res_dataset._predictions.list_models()
for model in models:
    test_pred = res_dataset._predictions.get_predictions(
        model=model, partition="test"
    )
    if test_pred:
        first_key = list(test_pred.keys())[0]
        pred_data = test_pred[first_key]
        y_true = pred_data['y_true'].flatten()
        y_pred = pred_data['y_pred'].flatten()
        mse = mean_squared_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        print(f"  {model}: MSE={mse:.4f}, R²={r2:.4f}")

print("\n✅ Prediction storage and analysis system is fully functional!")

Prediction Analysis Example:
PLSRegression predictions: ['sample_data_config_demo_pipeline_00090c_PLSRegression_test', 'sample_data_config_demo_pipeline_00090c_PLSRegression_test_fold_0', 'sample_data_config_demo_pipeline_00090c_PLSRegression_test_fold_1', 'sample_data_config_demo_pipeline_00090c_PLSRegression_test_fold_2']

PLS Test Set Performance:
  MSE: 15.1360
  MAE: 1.1331
  R²:  0.9624
  Y processing: numeric_StandardScaler5

PLS Cross-Validation Combined Performance:
  Samples: 99
  MSE: 415.0967
  MAE: 14.9935
  R²:  0.2374
  Folds: 3

Model Comparison (Test Set):
  PLSRegression: MSE=15.1360, R²=0.9624
  TestModel: MSE=0.0200, R²=0.9700
  Sequential: MSE=446.0420, R²=-0.1083

✅ Prediction storage and analysis system is fully functional!


In [1]:
# Reset autoreload completely
try:
    %autoreload 0  # Disable autoreload
    %reload_ext autoreload
    %autoreload 2  # Re-enable with full reload
except:
    %load_ext autoreload
    %autoreload 2

from nirs4all.pipeline.runner import PipelineRunner
from nirs4all.pipeline.config import PipelineConfig
from nirs4all.dataset import dataset
from sample import dataset_config, pipeline_config
from nirs4all.dataset.loader import get_dataset
from nirs4all.controllers.registry import reset_registry
from nirs4all.controllers import *

data = get_dataset(dataset_config)
config = PipelineConfig(pipeline_config, "demo_pipeline")

runner = PipelineRunner()
res_dataset, history, pipeline = runner.run(config, data)


# json_config = PipelineConfig("sample.json")
# yaml_config = PipelineConfig("sample.yaml")

Registering controller: DummyController
Registering controller: SklearnModelController
Registering controller: TensorFlowModelController
Registering controller: PyTorchModelController
Registering controller: TransformerMixinController
Registering controller: YTransformerMixinController
Registering controller: FeatureAugmentationController
Registering controller: SampleAugmentationController
Registering controller: CrossValidatorController
Registering controller: SpectraChartController
Registering controller: FoldChartController
Registering controller: YChartController
Loading dataset:
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
⚠️ Dataset does not have data for train_group.
⚠️ Dataset does not have data for test_group.
📊 Dataset: sample_data
Features (samples=189, sources=1):
- Source 0: (189, 1, 2151), processings=['raw'], min=-0.

d:\Workspace\ML\NIRS\nirs4all\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🏆 Training with best parameters: {'n_components': 18}
(97, 6453) (97,) (33, 6453) (33, 1)
(97, 6453) (97,) (33, 6453) (33, 1)
(97, 6453) (97,) (33, 6453) (33, 1)
💾 Saved 5_finetuned_PLSRegression_6.pkl to results\sample_data\config_demo_pipeline_00090c\5_finetuned_PLSRegression_6.pkl
💾 Saved 5_predictions_finetuned_7.csv to results\sample_data\config_demo_pipeline_00090c\5_predictions_finetuned_7.csv
💾 Saved 5_trained_PLSRegression_8_simple_cv_fold1.pkl to results\sample_data\config_demo_pipeline_00090c\5_trained_PLSRegression_8_simple_cv_fold1.pkl
💾 Saved 5_predictions_trained_9_simple_cv_fold1.csv to results\sample_data\config_demo_pipeline_00090c\5_predictions_trained_9_simple_cv_fold1.csv
💾 Saved 5_trained_PLSRegression_10_simple_cv_fold2.pkl to results\sample_data\config_demo_pipeline_00090c\5_trained_PLSRegression_10_simple_cv_fold2.pkl
💾 Saved 5_predictions_trained_11_simple_cv_fold2.csv to results\sample_data\config_demo_pipeline_00090c\5_predictions_trained_11_simple_cv_fold2.

(97, 2151, 3) (97,) (33, 2151, 3) (33, 1)
💾 Saved 6_finetuned_Sequential_14.pkl to results\sample_data\config_demo_pipeline_00090c\6_finetuned_Sequential_14.pkl
💾 Saved 6_predictions_finetuned_15.csv to results\sample_data\config_demo_pipeline_00090c\6_predictions_finetuned_15.csv
💾 Saved 6_trained_Sequential_16_simple_cv_fold1.pkl to results\sample_data\config_demo_pipeline_00090c\6_trained_Sequential_16_simple_cv_fold1.pkl
💾 Saved 6_predictions_trained_17_simple_cv_fold1.csv to results\sample_data\config_demo_pipeline_00090c\6_predictions_trained_17_simple_cv_fold1.csv
💾 Saved 6_trained_Sequential_18_simple_cv_fold2.pkl to results\sample_data\config_demo_pipeline_00090c\6_trained_Sequential_18_simple_cv_fold2.pkl
💾 Saved 6_predictions_trained_19_simple_cv_fold2.csv to results\sample_data\config_demo_pipeline_00090c\6_predictions_trained_19_simple_cv_fold2.csv
💾 Saved 6_trained_Sequential_20_simple_cv_fold3.pkl to results\sample_data\config_demo_pipeline_00090c\6_trained_Sequential_20

In [ ]:
# Test the new Predictions functionality
print("Testing Predictions functionality...")
print("\nDataset before running:")
print(f"Predictions: {res_dataset._predictions}")

# Let's check if any predictions were stored
print(f"\nNumber of predictions stored: {len(res_dataset._predictions)}")
print(f"Prediction keys: {res_dataset._predictions.list_keys()}")

# Try to manually add a test prediction
import numpy as np

res_dataset._predictions.add_prediction(
    dataset="test_dataset",
    pipeline="test_pipeline",
    model="TestModel",
    partition="test",
    y_true=np.array([1.0, 2.0, 3.0]),
    y_pred=np.array([1.1, 1.9, 3.2]),
    metadata={"test": True}
)

print(f"\nAfter adding test prediction:")
print(f"Number of predictions: {len(res_dataset._predictions)}")
print(f"Prediction keys: {res_dataset._predictions.list_keys()}")

# Get the test prediction
test_pred = res_dataset._predictions.get_prediction_data(
    "test_dataset", "test_pipeline", "TestModel", "test"
)
print(f"\nTest prediction data: {test_pred}")

Testing Predictions functionality...

Dataset before running:
Predictions: 📈 Predictions: 8 entries
   Datasets: ['sample_data']
   Pipelines: ['config_demo_pipeline_00090c']
   Models: ['PLSRegression', 'function', 'Sequential']

Number of predictions stored: 8
Prediction keys: ['sample_data_config_demo_pipeline_00090c_PLSRegression_test', 'sample_data_config_demo_pipeline_00090c_PLSRegression_test_fold_0', 'sample_data_config_demo_pipeline_00090c_PLSRegression_test_fold_1', 'sample_data_config_demo_pipeline_00090c_PLSRegression_test_fold_2', 'sample_data_config_demo_pipeline_00090c_Sequential_test', 'sample_data_config_demo_pipeline_00090c_function_test_fold_0', 'sample_data_config_demo_pipeline_00090c_function_test_fold_1', 'sample_data_config_demo_pipeline_00090c_function_test_fold_2']

After adding test prediction:
Number of predictions: 9
Prediction keys: ['sample_data_config_demo_pipeline_00090c_PLSRegression_test', 'sample_data_config_demo_pipeline_00090c_PLSRegression_test_fol